<a href="https://colab.research.google.com/github/jeaneigsi/AI-Scientist/blob/main/pretraining/Bert_Pre_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

In [11]:
MAX_LEN = 5000
BATCH_SIZE = 16 # per TPU core
TOTAL_STEPS = 2000  # thats approx 4 epochs
EVALUATE_EVERY = 200
LR =  5e-4

PRETRAINED_MODEL = 'answerdotai/ModernBERT-base'


import os
import numpy as np
import pandas as pd
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.optimizers import Adam
import transformers
import logging

AUTO = tf.data.experimental.AUTOTUNE

2.15.0


In [2]:
def connect_to_TPU():
    """Detect hardware, return appropriate distribution strategy"""
    try:
        # TPU detection. No parameters necessary if TPU_NAME environment variable is
        # set: this is always the case on Kaggle.
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        print('Running on TPU ', tpu.master())
    except ValueError:
        tpu = None

    if tpu:
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
    else:
        # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
        strategy = tf.distribute.get_strategy()

    global_batch_size = BATCH_SIZE * strategy.num_replicas_in_sync

    return tpu, strategy, global_batch_size


tpu, strategy, global_batch_size = connect_to_TPU()
print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  


REPLICAS:  8


In [ ]:
!wget https://raw.githubusercontent.com/SrinidhiRaghavan/AI-Sentiment-Analysis-on-IMDB-Dataset/master/imdb_tr.csv

--2020-09-02 10:33:57--  https://raw.githubusercontent.com/SrinidhiRaghavan/AI-Sentiment-Analysis-on-IMDB-Dataset/master/imdb_tr.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23677025 (23M) [text/plain]
Saving to: ‘imdb_tr.csv’

imdb_tr.csv         100%[===================>]  22.58M  49.2MB/s    in 0.5s    

2020-09-02 10:33:58 (49.2 MB/s) - ‘imdb_tr.csv’ saved [23677025/23677025]



In [ ]:
!pip install datasets


In [5]:
# data = pd.read_csv('imdb_tr.csv', encoding = "ISO-8859-1")

from datasets import load_dataset
# get Croatian data
fw = load_dataset("HuggingFaceFW/fineweb-2", name="fra_Latn", split="train", streaming=True)

README.md:   0%|          | 0.00/844k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/116 [00:00<?, ?it/s]

In [6]:
# data.head()
next(iter(fw))

{'text': "\n\nL'association\n\n\nL'abeille noire\n\n\nMédias\n\n\nNous contacter\nL'acquisition de notre génétique est conditionnée par l'adhésion au Conservatoire pour l'année en vigueur (12€ pour l'année 2013).\nUne réservation n'est effective qu'après réception du règlement et est prise en compte dans l'ordre chronologique d'arrivée des règlements.\nL'encaissement n'est réalisé qu'après la livraison. .\n2-Les reines\n- Reines d'abeilles noires (mellifera mellifera) écotype breton élevées dans les ruchers de l'île de Ouessant et maintenues trois semaines minimum en nucléi afin de permettre un développement suffisant des phéromones ; du volume de ponte et vérifier la qualité de leur ponte.\n- Les Reines fécondées (fécondation dirigée) sont disponibles à partir du 20 ou 25 juillet pour les reines de l'année ou de début avril ou début mai (selon la météo) pour les reines hivernées. Date limite de livraison en fin de saison : le 15 septembre.\n- Expédition par colis Chronopost ou Colissi

In [ ]:
# dataset_head = fw.take(2000000)
# my_data=list(dataset_head)

In [9]:
from datasets import Dataset, DatasetDict

ds = Dataset.from_generator(lambda: (yield from dataset_head), features=dataset_head.features)
dd = DatasetDict({"train": ds})

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# from datasets import Dataset

# # Convert the iterable dataset to a list
# # data_list = list(iterable_ds)  # Replace iterable_ds with your dataset object

# # Create a Dataset directly from the list
# ds = Dataset.from_list(
#     my_data, # use iterable_ds.take(N) to load a subset instead
#     features=fw.features
# )

In [10]:
dd

DatasetDict({
    train: Dataset({
        features: ['text', 'id', 'dump', 'url', 'date', 'file_path', 'language', 'language_score', 'language_script', 'minhash_cluster_size', 'top_langs'],
        num_rows: 2000000
    })
})

In [12]:
ds=dd["train"]
ds

Dataset({
    features: ['text', 'id', 'dump', 'url', 'date', 'file_path', 'language', 'language_score', 'language_script', 'minhash_cluster_size', 'top_langs'],
    num_rows: 2000000
})

In [1]:
ds["text"][0]

NameError: name 'ds' is not defined

In [ ]:
#data = data.sample(1000)

In [ ]:
from transformers import AutoTokenizer

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=MAX_LEN)

# Load the pre-trained tokenizer
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)

# Apply the tokenization function to the dataset using map
encoded_dataset = ds.map(tokenize_function, batched=True)

# Get the input IDs as a NumPy array


In [ ]:
X_data = np.array(encoded_dataset["input_ids"])

In [ ]:
%%time

def regular_encode(texts, tokenizer, maxlen=5000):
    enc_di = tokenizer.batch_encode_plus(
        texts,
        return_attention_mask=False,
        return_token_type_ids=False,
        pad_to_max_length=True,
        max_length=maxlen,
        truncation=True
    )

    return np.array(enc_di['input_ids'])


tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
X_data = regular_encode(data.text.values, tokenizer, maxlen=MAX_LEN)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


CPU times: user 1min 4s, sys: 233 ms, total: 1min 4s
Wall time: 1min 5s


In [ ]:
def prepare_mlm_input_and_labels(X):
    # 30% BERT masking
    inp_mask = np.random.rand(*X.shape)<0.30
    # do not mask special tokens
    inp_mask[X<=2] = False
    # set targets to -1 by default, it means ignore
    labels =  -1 * np.ones(X.shape, dtype=int)
    # set labels for masked tokens
    labels[inp_mask] = X[inp_mask]

    # prepare input
    X_mlm = np.copy(X)
    # set input to [MASK] which is the last token for the 90% of tokens
    # this means leaving 10% unchanged
    inp_mask_2mask = inp_mask  & (np.random.rand(*X.shape)<0.90)
    X_mlm[inp_mask_2mask] = tokenizer.mask_token_id  # mask token is the last in the dict

    # set 10% to a random token
    inp_mask_2random = inp_mask_2mask  & (np.random.rand(*X.shape) < 1/9)
    X_mlm[inp_mask_2random] = np.random.randint(3, tokenizer.mask_token_id, inp_mask_2random.sum())

    return X_mlm, labels


# use validation and test data for mlm
X_train_mlm = np.vstack(X_data)
# masks and labels
X_train_mlm, y_train_mlm = prepare_mlm_input_and_labels(X_train_mlm)

In [ ]:
def create_dist_dataset(X, y=None, training=False):
    dataset = tf.data.Dataset.from_tensor_slices(X)

    ### Add y if present ###
    if y is not None:
        dataset_y = tf.data.Dataset.from_tensor_slices(y)
        dataset = tf.data.Dataset.zip((dataset, dataset_y))

    ### Repeat if training ###
    if training:
        dataset = dataset.shuffle(len(X)).repeat()

    dataset = dataset.batch(global_batch_size).prefetch(AUTO)

    ### make it distributed  ###
    dist_dataset = strategy.experimental_distribute_dataset(dataset)

    return dist_dataset


train_dist_dataset = create_dist_dataset(X_train_mlm, y_train_mlm, True)


In [ ]:
%%time

def create_mlm_model_and_optimizer():
    with strategy.scope():
        model = TFAutoModelWithLMHead.from_pretrained(PRETRAINED_MODEL)
        optimizer = tf.keras.optimizers.Adam(learning_rate=LR)
    return model, optimizer


mlm_model, optimizer = create_mlm_model_and_optimizer()
mlm_model.summary()

/usr/local/lib/python3.6/dist-packages/transformers/modeling_tf_auto.py:788: FutureWarning: The class `TFAutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `TFAutoModelForCausalLM` for causal language models, `TFAutoModelForMaskedLM` for masked language models and `TFAutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertForMaskedLM: ['nsp___cls']
- This IS expected if you are initializing TFBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForMaskedLM were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


Model: "tf_bert_for_masked_lm"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
mlm___cls (TFBertMLMHead)    multiple                  24459834  
Total params: 110,104,890
Trainable params: 110,104,890
Non-trainable params: 0
_________________________________________________________________
CPU times: user 14.5 s, sys: 15 s, total: 29.5 s
Wall time: 58.1 s


In [ ]:
def define_mlm_loss_and_metrics():
    with strategy.scope():
        mlm_loss_object = masked_sparse_categorical_crossentropy

        def compute_mlm_loss(labels, predictions):
            per_example_loss = mlm_loss_object(labels, predictions)
            loss = tf.nn.compute_average_loss(
                per_example_loss, global_batch_size = global_batch_size)
            return loss

        train_mlm_loss_metric = tf.keras.metrics.Mean()

    return compute_mlm_loss, train_mlm_loss_metric


def masked_sparse_categorical_crossentropy(y_true, y_pred):
    y_true_masked = tf.boolean_mask(y_true, tf.not_equal(y_true, -1))
    y_pred_masked = tf.boolean_mask(y_pred, tf.not_equal(y_true, -1))
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true_masked,
                                                          y_pred_masked,
                                                          from_logits=True)
    return loss



def train_mlm(train_dist_dataset, total_steps=2000, evaluate_every=200):
    step = 0
    ### Training lopp ###
    for tensor in train_dist_dataset:
        distributed_mlm_train_step(tensor)
        step+=1

        if (step % evaluate_every == 0):
            ### Print train metrics ###
            train_metric = train_mlm_loss_metric.result().numpy()
            print("Step %d, train loss: %.2f" % (step, train_metric))

            ### Reset  metrics ###
            train_mlm_loss_metric.reset_states()

        if step  == total_steps:
            break


@tf.function
def distributed_mlm_train_step(data):
    strategy.experimental_run_v2(mlm_train_step, args=(data,))


@tf.function
def mlm_train_step(inputs):
    features, labels = inputs

    with tf.GradientTape() as tape:
        predictions = mlm_model(features, training=True)[0]
        loss = compute_mlm_loss(labels, predictions)

    gradients = tape.gradient(loss, mlm_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, mlm_model.trainable_variables))

    train_mlm_loss_metric.update_state(loss)


compute_mlm_loss, train_mlm_loss_metric = define_mlm_loss_and_metrics()

In [ ]:
%%time
train_mlm(train_dist_dataset, TOTAL_STEPS, EVALUATE_EVERY)

Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.


Instructions for updating:
renamed to `run`


Instructions for updating:
renamed to `run`


Step 200, train loss: 8.89
Step 400, train loss: 8.03
Step 600, train loss: 7.68
Step 800, train loss: 7.43
Step 1000, train loss: 7.22
Step 1200, train loss: 7.00
Step 1400, train loss: 6.86
Step 1600, train loss: 6.68
Step 1800, train loss: 6.54
Step 2000, train loss: 6.38
CPU times: user 1min 23s, sys: 13.4 s, total: 1min 37s
Wall time: 9min 3s


In [ ]:
mlm_model.save_pretrained('modern_bert_french')

# Load and Test

In [ ]:
from transformers import *
from pprint import pprint

In [ ]:
pretrained_model = TFAutoModelWithLMHead.from_pretrained(PRETRAINED_MODEL)
nlp = pipeline("fill-mask",model=pretrained_model, tokenizer=tokenizer ,framework='tf')
pprint(nlp(f"I watched {nlp.tokenizer.mask_token} and that was awesome"))

/usr/local/lib/python3.6/dist-packages/transformers/modeling_tf_auto.py:788: FutureWarning: The class `TFAutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `TFAutoModelForCausalLM` for causal language models, `TFAutoModelForMaskedLM` for masked language models and `TFAutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertForMaskedLM: ['nsp___cls']
- This IS expected if you are initializing TFBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForMaskedLM

[{'score': 0.31239137053489685,
  'sequence': '[CLS] i watched him and that was awesome [SEP]',
  'token': 2032,
  'token_str': 'him'},
 {'score': 0.1729636937379837,
  'sequence': '[CLS] i watched her and that was awesome [SEP]',
  'token': 2014,
  'token_str': 'her'},
 {'score': 0.13816313445568085,
  'sequence': '[CLS] i watched it and that was awesome [SEP]',
  'token': 2009,
  'token_str': 'it'},
 {'score': 0.08374697715044022,
  'sequence': '[CLS] i watched, and that was awesome [SEP]',
  'token': 1010,
  'token_str': ','},
 {'score': 0.06438492983579636,
  'sequence': '[CLS] i watched them and that was awesome [SEP]',
  'token': 2068,
  'token_str': 'them'}]


In [ ]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')
unmasker("Hello I'm a [MASK] model.")

/usr/local/lib/python3.6/dist-packages/transformers/modeling_tf_auto.py:788: FutureWarning: The class `TFAutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `TFAutoModelForCausalLM` for causal language models, `TFAutoModelForMaskedLM` for masked language models and `TFAutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
All model checkpoint weights were used when initializing TFBertForMaskedLM.

All the weights of TFBertForMaskedLM were initialized from the model checkpoint at imdb_bert_uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


[{'score': 0.4467789828777313,
  'sequence': '[CLS] i watched it and that was awesome [SEP]',
  'token': 2009,
  'token_str': 'it'},
 {'score': 0.06318594515323639,
  'sequence': '[CLS] i watched movie and that was awesome [SEP]',
  'token': 3185,
  'token_str': 'movie'},
 {'score': 0.056345004588365555,
  'sequence': '[CLS] i watched, and that was awesome [SEP]',
  'token': 1010,
  'token_str': ','},
 {'score': 0.013144557364284992,
  'sequence': '[CLS] i watched this and that was awesome [SEP]',
  'token': 2023,
  'token_str': 'this'},
 {'score': 0.012886741198599339,
  'sequence': '[CLS] i watched one and that was awesome [SEP]',
  'token': 2028,
  'token_str': 'one'}]
